In [22]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import recall_score, classification_report
from sklearn.impute import SimpleImputer, KNNImputer

#defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'rachaeld-data445'
bucket = s3.Bucket(bucket_name)

#defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [23]:
##defining input and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

##splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, stratify = Y)

In [25]:
X_train.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3352.000000,3373.000000,3389.000000
mean,49.549263,0.497935,236.687053,25.832247,75.921806
std,8.533676,0.500069,44.693721,4.102776,12.085101
min,32.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,23.090000,68.000000
50%,49.000000,0.000000,234.000000,25.420000,75.000000
75%,56.000000,1.000000,263.000000,28.150000,83.000000
max,70.000000,1.000000,696.000000,56.800000,143.000000


In [26]:
##defining the imputer
imputer = KNNImputer(n_neighbors = 5, weights = 'distance').fit(X_train)

##imputing the missing values
X_train_imp = pd.DataFrame(imputer.transform(X_train), columns = X_train.columns)
X_test_imp = pd.DataFrame(imputer.transform(X_test), columns = X_test.columns)

In [27]:
X_train_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3390.000000,3390.000000,3390.000000
mean,49.549263,0.497935,236.635664,25.834106,75.921368
std,8.533676,0.500069,44.555635,4.098408,12.083345
min,32.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,23.090000,68.000000
50%,49.000000,0.000000,234.000000,25.416401,75.000000
75%,56.000000,1.000000,262.000000,28.147500,83.000000
max,70.000000,1.000000,696.000000,56.800000,143.000000


In [28]:
X_test_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,848.000000,848.000000,848.000000
mean,49.727594,0.478774,236.968401,25.675903,75.707547
std,8.727954,0.499844,43.930866,3.985698,11.795356
min,34.000000,0.000000,113.000000,16.590000,47.000000
25%,42.000000,0.000000,206.000000,22.962500,68.000000
50%,49.000000,0.000000,233.000000,25.225000,75.000000
75%,57.000000,1.000000,265.000000,27.730000,82.000000
max,70.000000,1.000000,439.000000,45.800000,122.000000


In [29]:
### RANDOM FOREST ###
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_imp, Y_train)

##predicting on test
RF_pred = RF_md.predict_proba(X_test_imp)[:,1]

##changing liklihoods to labels
RF_label = np.where(RF_pred < .1, 0, 1)

print(classification_report(Y_test, RF_label))

              precision    recall  f1-score   support

           0       0.92      0.38      0.54       719
           1       0.19      0.83      0.31       129

    accuracy                           0.45       848
   macro avg       0.56      0.60      0.42       848
weighted avg       0.81      0.45      0.50       848



In [30]:
### ADABOOST ### 
ADA_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier( max_depth = 5), n_estimators = 500, learning_rate = .01).fit(X_train_imp, Y_train)

##predicting on test
ADA_pred = ADA_md.predict_proba(X_test_imp)[:,1]

##changing liklihoods to labels
ADA_label = np.where(ADA_pred < .1, 0, 1)

print(classification_report(Y_test, ADA_label))

              precision    recall  f1-score   support

           0       0.88      0.16      0.27       719
           1       0.16      0.88      0.27       129

    accuracy                           0.27       848
   macro avg       0.52      0.52      0.27       848
weighted avg       0.77      0.27      0.27       848



In [ ]:
## based on the above results I would use my Random Forest model to predict TenYearCHD, the liklihood of coronary disease,
## because it has overall the better scores. 